<a href="https://colab.research.google.com/github/Mohammadhsiavash/DeepL-Training/blob/main/Computer%20Vision/Emotion_Detection_from_Faces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Build a deep learning model to classify human emoons from facial expressions in
images.


In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("msambare/fer2013")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/fer2013


## Load the dataset


In [2]:
import os
for dirname, _, filenames in os.walk('/'):
    for filename in filenames:
        if 'fer2013.csv' in filename:
            print(os.path.join(dirname, filename))

In [3]:
import subprocess

# Use shell command to list files
result = subprocess.run(['ls', '-R', '/kaggle/input/fer2013'], capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

/kaggle/input/fer2013:
test
train

/kaggle/input/fer2013/test:
angry
disgust
fear
happy
neutral
sad
surprise

/kaggle/input/fer2013/test/angry:
PrivateTest_10131363.jpg
PrivateTest_10304478.jpg
PrivateTest_1054527.jpg
PrivateTest_10590091.jpg
PrivateTest_1109992.jpg
PrivateTest_11296953.jpg
PrivateTest_12000629.jpg
PrivateTest_12008383.jpg
PrivateTest_12191716.jpg
PrivateTest_1221822.jpg
PrivateTest_12403575.jpg
PrivateTest_12766285.jpg
PrivateTest_12846357.jpg
PrivateTest_1290484.jpg
PrivateTest_12912780.jpg
PrivateTest_13164119.jpg
PrivateTest_13278552.jpg
PrivateTest_13463625.jpg
PrivateTest_13541561.jpg
PrivateTest_13664224.jpg
PrivateTest_14325168.jpg
PrivateTest_14426977.jpg
PrivateTest_14433751.jpg
PrivateTest_14444117.jpg
PrivateTest_14482875.jpg
PrivateTest_14494003.jpg
PrivateTest_14565821.jpg
PrivateTest_14596578.jpg
PrivateTest_14740105.jpg
PrivateTest_14769246.jpg
PrivateTest_1488292.jpg
PrivateTest_14887556.jpg
PrivateTest_15489614.jpg
PrivateTest_15827448.jpg
PrivateTest

In [4]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Streaming output truncated to the last 5000 lines.
/kaggle/input/fer2013/train/happy/Training_71717074.jpg
/kaggle/input/fer2013/train/happy/Training_83349364.jpg
/kaggle/input/fer2013/train/happy/Training_8224205.jpg
/kaggle/input/fer2013/train/happy/Training_69103613.jpg
/kaggle/input/fer2013/train/happy/Training_7098848.jpg
/kaggle/input/fer2013/train/happy/Training_38129729.jpg
/kaggle/input/fer2013/train/happy/Training_95151390.jpg
/kaggle/input/fer2013/train/happy/Training_66874071.jpg
/kaggle/input/fer2013/train/happy/Training_23294249.jpg
/kaggle/input/fer2013/train/happy/Training_86508875.jpg
/kaggle/input/fer2013/train/happy/Training_50691325.jpg
/kaggle/input/fer2013/train/happy/Training_26593901.jpg
/kaggle/input/fer2013/train/happy/Training_81451115.jpg
/kaggle/input/fer2013/train/happy/Training_39982194.jpg
/kaggle/input/fer2013/train/happy/Training_18980574.jpg
/kaggle/input/fer2013/train/happy/Training_39446205.jpg
/kaggle/input/fer2013/train/happy/Training_9457008.jpg


## Load images and labels

Load the images from the train and test directories and create corresponding labels based on the directory names.


In [5]:
import os

base_path = '/kaggle/input/fer2013'
train_images = []
train_labels = []
test_images = []
test_labels = []
emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

for emotion in emotions:
    train_dir = os.path.join(base_path, 'train', emotion)
    test_dir = os.path.join(base_path, 'test', emotion)

    for filename in os.listdir(train_dir):
        train_images.append(os.path.join(train_dir, filename))
        train_labels.append(emotion)

    for filename in os.listdir(test_dir):
        test_images.append(os.path.join(test_dir, filename))
        test_labels.append(emotion)

print(f"Number of training images: {len(train_images)}")
print(f"Number of testing images: {len(test_images)}")
print(f"First 5 training image paths: {train_images[:5]}")
print(f"First 5 training labels: {train_labels[:5]}")
print(f"First 5 testing image paths: {test_images[:5]}")
print(f"First 5 testing labels: {test_labels[:5]}")

Number of training images: 28709
Number of testing images: 7178
First 5 training image paths: ['/kaggle/input/fer2013/train/angry/Training_52322132.jpg', '/kaggle/input/fer2013/train/angry/Training_93516694.jpg', '/kaggle/input/fer2013/train/angry/Training_59490301.jpg', '/kaggle/input/fer2013/train/angry/Training_20556787.jpg', '/kaggle/input/fer2013/train/angry/Training_98419709.jpg']
First 5 training labels: ['angry', 'angry', 'angry', 'angry', 'angry']
First 5 testing image paths: ['/kaggle/input/fer2013/test/angry/PublicTest_3641536.jpg', '/kaggle/input/fer2013/test/angry/PublicTest_21334851.jpg', '/kaggle/input/fer2013/test/angry/PrivateTest_56252993.jpg', '/kaggle/input/fer2013/test/angry/PrivateTest_46881419.jpg', '/kaggle/input/fer2013/test/angry/PrivateTest_88795628.jpg']
First 5 testing labels: ['angry', 'angry', 'angry', 'angry', 'angry']


## Preprocess images

Resize and normalize the images to a consistent format suitable for a deep learning model.


In [6]:
import cv2
import numpy as np

def preprocess_image(image_path):
    """Reads, resizes, and normalizes an image."""
    img = cv2.imread(image_path)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    resized_img = cv2.resize(gray_img, (48, 48))
    normalized_img = resized_img / 255.0
    return normalized_img

train_images_processed = [preprocess_image(img_path) for img_path in train_images]
test_images_processed = [preprocess_image(img_path) for img_path in test_images]

train_images_processed = np.array(train_images_processed)
test_images_processed = np.array(test_images_processed)

print("Shape of processed training images:", train_images_processed.shape)
print("Shape of processed testing images:", test_images_processed.shape)

Shape of processed training images: (28709, 48, 48)
Shape of processed testing images: (7178, 48, 48)


## Encode labels


Convert the emotion labels into a one-hot encoded format.


In [7]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform training and testing labels
all_labels = train_labels + test_labels
label_encoder.fit(all_labels)

train_labels_encoded = label_encoder.transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Convert to one-hot encoded format
train_labels_one_hot = to_categorical(train_labels_encoded)
test_labels_one_hot = to_categorical(test_labels_encoded)

# Print shapes to verify
print("Shape of one-hot encoded training labels:", train_labels_one_hot.shape)
print("Shape of one-hot encoded testing labels:", test_labels_one_hot.shape)

Shape of one-hot encoded training labels: (28709, 7)
Shape of one-hot encoded testing labels: (7178, 7)


## Build the model

Design and compile a deep learning model (e.g., a CNN) for image classification.


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the number of emotion classes
num_classes = len(emotions)

# Build the Sequential model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 46, 46, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 21, 21, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 355,847 (1.36 MB)

 Trainable params: 355,847 (1.36 MB)

 Non-trainable params: 0 (0.00 B)

## Train the model

Train the model using the preprocessed images and encoded labels.


In [9]:
history = model.fit(
    train_images_processed,
    train_labels_one_hot,
    epochs=15,
    batch_size=64,
    validation_data=(test_images_processed, test_labels_one_hot)
)

Epoch 1/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 12s 13ms/step - accuracy: 0.3061 - loss: 1.7189 - val_accuracy: 0.4543 - val_loss: 1.4348
Epoch 2/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.4623 - loss: 1.4025 - val_accuracy: 0.4957 - val_loss: 1.3081
Epoch 3/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.5256 - loss: 1.2559 - val_accuracy: 0.5173 - val_loss: 1.2636
Epoch 4/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.5632 - loss: 1.1644 - val_accuracy: 0.5382 - val_loss: 1.2053
Epoch 5/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.5909 - loss: 1.0970 - val_accuracy: 0.5535 - val_loss: 1.1798
Epoch 6/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.6128 - loss: 1.0363 - val_accuracy: 0.5600 - val_loss: 1.1573
Epoch 7/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.6466 - loss: 0.9565 - val_accuracy: 0.5659 - val_loss: 1.1671
Epoch 8/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.6646 - loss: 0.9056 - val_accuracy: 

## Evaluate the model

Evaluate the trained model's performance on the test set.


In [10]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_images_processed, test_labels_one_hot, verbose=0)

# Print the evaluation results
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Loss: 1.5720
Test Accuracy: 0.5652


## Summary:

### Data Analysis Key Findings

*   The FER-2013 dataset contains 28,709 training images and 7,178 testing images across 7 emotion categories.
*   Images were successfully preprocessed by converting to grayscale, resizing to 48x48 pixels, and normalizing pixel values.
*   Emotion labels were one-hot encoded into a format suitable for model training.
*   A Convolutional Neural Network (CNN) model was built and compiled for emotion classification.
*   The model was trained for 15 epochs with a batch size of 64.
*   Upon evaluation on the test set, the model achieved a Test Loss of 1.5720 and a Test Accuracy of 0.5652.

### Insights or Next Steps

*   The achieved test accuracy of 56.52% suggests that the model has learned to some extent, but there is significant room for improvement.
*   Further steps could include exploring more complex model architectures, employing data augmentation techniques, or fine-tuning hyperparameters to potentially increase accuracy.
